## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Butaritari,KI,3.07,172.79,79.86,76,80,4.23,broken clouds
1,1,Kahului,US,20.89,-156.47,73.42,83,40,8.05,light rain
2,2,Kapaa,US,22.08,-159.32,80.19,82,100,21.85,light rain
3,3,Georgetown,MY,5.41,100.34,83.89,86,20,3.44,few clouds
4,4,Rikitea,PF,-23.12,-134.97,74.32,73,21,10.07,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 88


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Butaritari,KI,3.07,172.79,79.86,76,80,4.23,broken clouds
2,2,Kapaa,US,22.08,-159.32,80.19,82,100,21.85,light rain
3,3,Georgetown,MY,5.41,100.34,83.89,86,20,3.44,few clouds
8,8,Mogadishu,SO,2.04,45.34,76.42,84,40,18.25,scattered clouds
10,10,Hasaki,JP,35.73,140.83,79.29,91,100,13.60,light rain
13,13,Puerto Del Rosario,ES,28.50,-13.86,80.35,69,0,17.27,clear sky
14,14,Mao,ES,39.89,4.27,86.56,51,0,10.36,clear sky
22,22,Samarai,PG,-10.62,150.67,79.88,82,1,8.84,clear sky
37,37,Cayenne,GF,4.93,-52.33,86.09,22,21,8.05,few clouds
39,39,Ferkessedougou,CI,9.60,-5.20,83.84,57,93,3.24,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                191
City                   191
Country                191
Lat                    191
Lng                    191
Max Temp               191
Humidity               191
Cloudiness             191
Wind Speed             191
Current Description    191
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)].dropna()
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Butaritari,KI,3.07,172.79,79.86,76,80,4.23,broken clouds
2,2,Kapaa,US,22.08,-159.32,80.19,82,100,21.85,light rain
3,3,Georgetown,MY,5.41,100.34,83.89,86,20,3.44,few clouds
8,8,Mogadishu,SO,2.04,45.34,76.42,84,40,18.25,scattered clouds
10,10,Hasaki,JP,35.73,140.83,79.29,91,100,13.60,light rain
13,13,Puerto Del Rosario,ES,28.50,-13.86,80.35,69,0,17.27,clear sky
14,14,Mao,ES,39.89,4.27,86.56,51,0,10.36,clear sky
22,22,Samarai,PG,-10.62,150.67,79.88,82,1,8.84,clear sky
37,37,Cayenne,GF,4.93,-52.33,86.09,22,21,8.05,few clouds
39,39,Ferkessedougou,CI,9.60,-5.20,83.84,57,93,3.24,overcast clouds


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Butaritari,KI,79.86,broken clouds,3.07,172.79,
2,Kapaa,US,80.19,light rain,22.08,-159.32,
3,Georgetown,MY,83.89,few clouds,5.41,100.34,
8,Mogadishu,SO,76.42,scattered clouds,2.04,45.34,
10,Hasaki,JP,79.29,light rain,35.73,140.83,
13,Puerto Del Rosario,ES,80.35,clear sky,28.50,-13.86,
14,Mao,ES,86.56,clear sky,39.89,4.27,
22,Samarai,PG,79.88,clear sky,-10.62,150.67,
37,Cayenne,GF,86.09,few clouds,4.93,-52.33,
39,Ferkessedougou,CI,83.84,overcast clouds,9.60,-5.20,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
            
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].replace('', pd.NA, inplace=True)
final_hotel_df = hotel_df.dropna()
final_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Butaritari,KI,79.86,broken clouds,3.07,172.79,Isles Sunset Lodge
2,Kapaa,US,80.19,light rain,22.08,-159.32,Sheraton Kauai Coconut Beach Resort
3,Georgetown,MY,83.89,few clouds,5.41,100.34,Cititel Penang
8,Mogadishu,SO,76.42,scattered clouds,2.04,45.34,Hotel Juba
10,Hasaki,JP,79.29,light rain,35.73,140.83,Inubosaki Kanko Hotel
...,...,...,...,...,...,...,...
517,Oxford,GB,75.38,broken clouds,51.75,-1.26,Malmaison Oxford
518,Anqing,CN,81.55,broken clouds,30.51,117.05,Hongtian Hotel
525,Vyartsilya,RU,78.62,few clouds,62.18,30.69,Erakkoranta
526,Trairi,BR,82.98,overcast clouds,-3.28,-39.27,janelas do mar


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
final_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in final_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = final_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))